In [1]:
import json, os
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import math
import warnings

## Data Collection

In [2]:
tickers = ['NVDA', 'GOOG', 'AMZN','AAPL', 'MSFT', 'SPOT', 'TSLA', 'JPM', 'GS', 'LMT']
start_date = "2021-10-15"
end_date = "2025-10-17"

data = yf.download(tickers, start = start_date, end = end_date, group_by= "ticker", interval = "1d")
vix = yf.download("^VIX", start = start_date, end = end_date, group_by= "ticker", interval = "1d")

stocks_csv_path = f"../data/market_data/stocks_{start_date}_{end_date}.csv"
vix_csv_path = f"../data/market_data/vix_{start_date}_{end_date}.csv"

data.to_csv(stocks_csv_path)
vix.to_csv(vix_csv_path)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
tickers = ['NVDA', 'GOOG', 'AMZN', 'AAPL', 'MSFT', 'SPOT', 'TSLA', 'JPM', 'GS', 'LMT']
start_date = "2021-10-15"
end_date = "2025-10-17"
df = pd.read_csv(f"../data/market_data/stocks_{start_date}_{end_date}.csv", header=[0, 1], index_col=0)
df.head()

Ticker            AMZN                                                 \
Price             Open        High         Low       Close     Volume   
Date                                                                    
2021-10-15  165.570999  170.520996  165.199997  170.451004  103598000   
2021-10-18  169.417999  172.458496  169.255005  172.337006   63482000   
2021-10-19  171.714493  172.734497  171.100006  172.207504   47722000   
2021-10-20  172.632996  173.143005  170.018494  170.753006   42796000   
2021-10-21  170.712494  172.014008  170.149994  171.750504   37628000   

Ticker            GOOG                                                ...  \
Price             Open        High         Low       Close    Volume  ...   
Date                                                                  ...   
2021-10-15  141.233492  141.233492  140.105712  140.712067  21250000  ...   
2021-10-18  140.253710  142.026824  140.253710  141.988831  16564000  ...   
2021-10-19  142.317601  143.127549  142.123419  142.844498  15316000  ...   
2021-10-20  143.242262  143.267344  140.947465  141.447037  17940000  ...   
2021-10-21  141.225573  141.878605  140.674337  141.810074  14850000  ...   

Ticker             JPM                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2021-10-15  148.869466  150.508058  147.780061  150.003876  14267400   
2021-10-18  149.886799  151.804505  149.373619  149.949829  12672900   
2021-10-19  150.715112  151.840523  149.976849  151.768509   8053900   
2021-10-20  151.525440  153.839280  150.769167  153.812271   8185600   
2021-10-21  153.740230  154.253424  151.867543  152.605820   8415200   

Ticker             LMT                                               
Price             Open        High         Low       Close   Volume  
Date                                                                 
2021-10-15  326.746818  328.667260  326.217349  328.110870   952700  
2021-10-18  326.953197  330.139018  324.754561  327.554474  1014600  
2021-10-19  328.900541  333.100411  326.342919  333.001709  1051300  
2021-10-20  333.638914  334.347875  331.862033  333.055603   955200  
2021-10-21  333.504308  334.967090  329.474948  334.859406   803600  

[5 rows x 50 columns]

In [4]:
close = df.xs('Close', axis=1, level=1)
volume = df.xs('Volume', axis=1, level=1)

close.columns = [f"{ticker}_Close" for ticker in close.columns]
volume.columns = [f"{ticker}_Volume" for ticker in volume.columns]

#Reset index to turn Date into a regular column
data = pd.concat([close, volume], axis=1)
data = data.reset_index()
data['Date'] = pd.to_datetime(data['Date'])

data

,Date,SPOT_Close,GS_Close,TSLA_Close,AMZN_Close,GOOG_Close,NVDA_Close,MSFT_Close,AAPL_Close,LMT_Close,...,SPOT_Volume,GS_Volume,TSLA_Volume,AMZN_Volume,GOOG_Volume,NVDA_Volume,MSFT_Volume,AAPL_Volume,LMT_Volume,JPM_Volume
0,2021-10-15,247.399994,366.803650,281.010010,170.451004,140.712082,21.818275,294.360779,141.657318,328.110931,...,813300,6451400,56773800,103598000,21250000,226994000,25384800,67940300,952700,14267400
1,2021-10-18,248.820007,373.686768,290.036682,172.337006,141.988831,22.177559,297.340973,143.329681,327.554474,...,763500,3832300,72621600,63482000,16564000,189494000,22729300,85589200,1014600,12672900
2,2021-10-19,255.610001,372.304718,288.089996,172.207504,142.844482,22.245420,298.250702,145.491150,333.001770,...,1056000,2568300,52143300,47722000,15316000,161473000,17682100,76378900,1051300,8053900
3,2021-10-20,251.179993,368.447632,288.600006,170.753006,141.447021,22.058794,297.457184,145.980148,333.055634,...,736700,2453400,42096300,42796000,17940000,146276000,16537100,58418800,955200,8185600
4,2021-10-21,255.000000,368.176636,298.000000,171.750504,141.810059,22.646618,300.698730,146.195328,334.859436,...,1093500,2162100,94444500,37628000,14850000,187590000,16918100,61421000,803600,8415200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,2025-10-10,685.289978,764.359985,413.489990,216.369995,237.490005,183.160004,510.959991,245.032471,505.049988,...,1377300,2457900,112107900,72367500,22449100,268774400,24133800,61999100,1313800,8597400
1001,2025-10-13,692.039978,786.780029,435.899994,220.070007,244.639999,188.320007,514.049988,247.420151,503.829987,...,1039000,2766000,79552800,37809700,12465000,153482800,14284200,38142900,926700,10788700
1002,2025-10-14,683.510010,770.760010,429.239990,216.389999,246.190002,180.029999,513.570007,247.530045,505.179993,...,1214600,4710800,72669400,45665600,17705400,205641400,14684300,35478000,1078000,16178800
1003,2025-10-15,671.299988,767.929993,435.149994,215.570007,251.710007,179.830002,513.429993,249.098526,499.410004,...,1103700,3217400,71558200,45909500,16048100,214450500,14694700,33893600,1678500,11354800


Now, we compute technical indicators and add sentiment indicators for the stocks selected. We also include a new column indicating the earnings date.

In [5]:
from tqdm import tqdm
dfs = {}

for ticker in tqdm(tickers, desc="Processing tickers"):

     # Retrieve the closing prices and create a new dataframe
    close = data[f"{ticker}_Close"].copy()
    volume = data[f"{ticker}_Volume"].copy()
    df = pd.DataFrame(index=close.index)
    df["Date"] = data['Date']
    df["Close"] = close
    df["Volume"] = volume
    
    # Compute daily returns and indicate whether each is positive or negative
    returns = close.pct_change()
    df["daily_ret"] = returns
    df["positive"] = (returns > 0).astype(int)
    for lag in range(1, 6):
        df[f"positive_lag{lag}"] = df["positive"].shift(lag)
    df["positive_next_day"] = df["positive"].shift(-1)
    
    for lag in range(1, 6):
        df[f'lag{lag}'] = returns.shift(lag)
    
    # Compute 5-day, 20-day and 50-day moving averages of closing prices
    df["MA5"] = close.rolling(window=5).mean()
    df["MA20"] = close.rolling(window=20).mean()
    df["MA50"] = close.rolling(window=50).mean()

     # Compute MACD
    short_ema = close.ewm(span=12, adjust=False).mean()
    long_ema = close.ewm(span=26, adjust=False).mean()
    df["MACD"] = short_ema - long_ema
    df["MACD_signal_line"] = df["MACD"].ewm(span=9, adjust=False).mean()

    # Compute the rolling volatility 
    df["vol5"] = returns.rolling(window=5).std()
    df["vol20"] = returns.rolling(window=20).std()
    df["vol50"] = returns.rolling(window=50).std()

    # Compute the 14-day RSI
    delta = close.diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / loss
    df["RSI_14"] = 100 - (100 / (1 + rs))


    # Add earnings date
    earning_df = yf.Ticker(ticker).get_earnings_dates(limit=4 * 3)
    if earning_df is not None and not earning_df.empty:
        earnings_dates = earning_df.index.tz_localize(None).normalize()
    else: 
        earnings_dates = []
    
    df["earnings"] = df.index.isin(earnings_dates)
    df["earnings_next_day"] = df["earnings"].shift(-1)

    # Save into dictionary
    dfs[ticker] = df.copy()

Processing tickers: 100%|██████████| 10/10 [00:00<00:00, 10.12it/s]


Now, we include pre-computed sentiment scores derived from market news. The sentiment for each trading day is calculated using news released after 4:00 PM of the previous day and up to 3:59 PM of the current day, since we aim to predict that day's closing price movement. In addition to company-specific news, we also retrieve the VIX index as an indicator of overall market sentiment and include it as a column in the training data.

In [6]:
vix = pd.read_csv(f"../market_data/vix_{start_date}_{end_date}.csv", skiprows=[0, 1])
vix.columns = ['Date', 'vix_open', 'vix_high','vix_low','vix_close','vix_volume']
vix

,Date,vix_open,vix_high,vix_low,vix_close,vix_volume
0,2021-10-15,16.639999,16.850000,15.720000,16.299999,0
1,2021-10-18,17.290001,17.930000,16.270000,16.309999,0
2,2021-10-19,16.090000,16.309999,15.570000,15.700000,0
3,2021-10-20,15.820000,15.890000,15.290000,15.490000,0
4,2021-10-21,16.059999,16.110001,14.920000,15.010000,0
...,...,...,...,...,...,...
1000,2025-10-10,16.360001,22.440001,16.230000,21.660000,0
1001,2025-10-13,19.450001,20.770000,18.610001,19.030001,0
1002,2025-10-14,21.459999,22.940001,19.180000,20.809999,0
1003,2025-10-15,20.020000,22.440001,19.110001,20.639999,0


In [7]:
tech_features = ['Date','earnings_next_day']#["Date", "Close", "Volume", "earnings_next_day", "vol5","vol20","MA5", "MA20", "MACD"]
sent_features = ["sentiment_mean"]
macros = ["vix_close"]
lagged_features = []#["daily_ret"] + [f"lag{lag}" for lag in range(1, 6)]
positives = ["positive_next_day"]
features = tech_features + sent_features + macros + lagged_features + positives

merged_dfs, train_dfs = {}, {}

for ticker in tickers:
    tech = dfs[ticker].copy()
    sent = pd.read_csv(f"../sentiment_scores/{ticker}_sentiment.csv")
    vix_close = vix[['Date','vix_close']].copy()
    
    tech["Date"] = pd.to_datetime(tech["Date"]).dt.date
    sent["Date"] = pd.to_datetime(sent["Date"]).dt.date
    vix_close["Date"] = pd.to_datetime(vix_close["Date"]).dt.date
    
    # Merge and select relevant features
    merged = (tech.merge(sent, on="Date", how="left").merge(vix_close, on="Date", how="left"))
    merged_dfs[ticker] = merged
    train_dfs[ticker] = merged[features].dropna(subset=features, how="any")

# Example: training sample for NVDA
training_sample = train_dfs.get("NVDA", pd.DataFrame())
training_size = len(training_sample)
training_sample

,Date,earnings_next_day,sentiment_mean,vix_close,positive_next_day
1,2021-10-18,False,0.000000,16.309999,1.0
2,2021-10-19,False,1.000000,15.700000,0.0
3,2021-10-20,False,0.833333,15.490000,1.0
4,2021-10-21,False,1.000000,15.010000,1.0
5,2021-10-22,False,1.000000,15.430000,1.0
...,...,...,...,...,...
999,2025-10-09,False,0.137931,16.430000,0.0
1000,2025-10-10,False,0.444444,21.660000,1.0
1001,2025-10-13,False,0.458333,19.030001,0.0
1002,2025-10-14,False,0.882353,20.809999,0.0


## Preditive Modeling

We train machine learning models that use technical indicators and sentiment scores to predict next-day price movements. The models include **K-Nearest Neighbors (KNN), Logistic Regression**, and **Support Vector Machines (SVM)**. For each model, we apply time-series cross-validation to identify the optimal hyperparameters. 

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import joblib

### K-nearest-neighbors (KNN)

In [9]:
model_dir = "../models"
os.makedirs(model_dir, exist_ok=True)

#warnings.filterwarnings("ignore")
X_trains, X_tests, y_trains, y_tests, train_pred_knn, test_pred_knn, k_vals = (
    {}, {}, {}, {}, {}, {}, {}
)

for ticker in tickers:
    df = train_dfs[ticker]
    # Features and target
    X = df.drop(columns=["Date", "positive_next_day"])
    y = df["positive_next_day"]

    # Train/test split (80/20)
    split_idx = int(len(df) * 0.8)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    X_trains[ticker], X_tests[ticker] = X_train, X_test
    y_trains[ticker], y_tests[ticker] = y_train, y_test

    # Pipeline
    pipe = Pipeline([("scaler", StandardScaler()),("knn", KNeighborsClassifier())])

    # Time-series CV grid search for best k
    tscv = TimeSeriesSplit(n_splits = 5)
    grid = {"knn__n_neighbors": list(range(3, 40)),
            "knn__weights": ["uniform", "distance"],}
    search = GridSearchCV(pipe, grid, cv=tscv, scoring="neg_mean_squared_error", n_jobs=-1)
    search.fit(X_train, y_train)

    best_k = search.best_params_["knn__n_neighbors"]
    k_vals[ticker] = best_k

    # Fit final model
    final_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier(n_neighbors=best_k))
    ])
    final_pipe.fit(X_train, y_train)

    model_path = os.path.join(model_dir, f"{ticker}_knn.pkl")
    joblib.dump(final_pipe, model_path)

    # Predictions
    train_pred_knn[ticker] = final_pipe.predict(X_train)
    test_pred_knn[ticker] = final_pipe.predict(X_test)

    print(f"{ticker}: best_k = {best_k}, train size = {len(X_train)}, test size = {len(X_test)}")

NVDA: best_k = 35, train size = 792, test size = 198
GOOG: best_k = 11, train size = 796, test size = 199
AMZN: best_k = 3, train size = 802, test size = 201
AAPL: best_k = 37, train size = 800, test size = 201
MSFT: best_k = 21, train size = 792, test size = 198
SPOT: best_k = 36, train size = 482, test size = 121
TSLA: best_k = 23, train size = 800, test size = 201
JPM: best_k = 31, train size = 700, test size = 175
GS: best_k = 19, train size = 626, test size = 157
LMT: best_k = 28, train size = 616, test size = 154


In [10]:
knn = pd.DataFrame({
    'Ticker': tickers,
    'K_value': list(k_vals.values()),
    'train_pred_accuracy': [accuracy_score(y_trains[t], train_pred_knn[t]) for t in tickers],
    'test_pred_accuracy': [accuracy_score(y_tests[t], test_pred_knn[t]) for t in tickers],
    'baseline (always positive)': [y_tests[t].mean() for t in tickers]
}).set_index('Ticker')
knn = knn.astype(float)
knn = knn.round(3)

n_total = len(knn)
above_50 = (knn['test_pred_accuracy'] >= 0.5).sum()
better_than_baseline = (knn['test_pred_accuracy'] > knn['baseline (always positive)']).sum()

print(f"Out of {n_total} tickers:")
print(f"{above_50} achieved test accuracy > 50%")
print(f"{better_than_baseline} performed better than the always-up baseline\n")
display(knn)

Out of 10 tickers:
10 achieved test accuracy > 50%
7 performed better than the always-up baseline



,K_value,train_pred_accuracy,test_pred_accuracy,baseline (always positive)
Ticker,,,,
NVDA,35.0,0.610,0.576,0.551
GOOG,11.0,0.626,0.538,0.533
AMZN,3.0,0.752,0.512,0.502
AAPL,37.0,0.590,0.532,0.512
MSFT,21.0,0.585,0.500,0.535
SPOT,36.0,0.568,0.521,0.545
TSLA,23.0,0.629,0.542,0.493
JPM,31.0,0.579,0.606,0.566
GS,19.0,0.609,0.561,0.516


### Logistic Regression

In [11]:
warnings.filterwarnings("ignore")
model_dir = "../models"
os.makedirs(model_dir, exist_ok=True)
results = []

for ticker in tickers:
    df = train_dfs[ticker]

    # Features and target
    X = df.drop(columns=["Date", "positive_next_day"])
    y = df["positive_next_day"]

    # Train/test split (80/20)
    split_idx = int(len(df) * 0.8)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    
    tscv = TimeSeriesSplit(n_splits=5)
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression(max_iter=1000, solver="lbfgs"))
    ])

    # Hyperparameter grid
    grid = {
        "logreg__C": [0.01, 0.1, 1, 5, 10, 20],
        "logreg__penalty": ["l2"],
        "logreg__class_weight": [None, "balanced"]
    }

    # Grid search
    search = GridSearchCV(
        estimator=pipe,
        param_grid=grid,
        cv=tscv,
        scoring="roc_auc",
        n_jobs=-1
    )
    search.fit(X_train, y_train)

    best_model = search.best_estimator_

    # Save the model
    model_path = os.path.join(model_dir, f"{ticker}_log.pkl")
    joblib.dump(best_model, model_path)

    # Predictions
    train_pred = best_model.predict(X_train)
    test_pred = best_model.predict(X_test)

    # Metrics
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    baseline = y_test.mean()

    results.append({
        "Ticker": ticker,
        "train_pred_accuracy": round(train_acc, 3),
        "test_pred_accuracy": round(test_acc, 3),
        "baseline (always positive)": round(baseline, 3)
    })

logreg = pd.DataFrame(results).set_index("Ticker")

# print statistics
n_total = len(logreg)
above_50 = (logreg['test_pred_accuracy'] >= 0.5).sum()
better_than_baseline = (logreg['test_pred_accuracy'] >= logreg['baseline (always positive)']).sum()

print(f"Out of {n_total} tickers:")
print(f"{above_50} achieved test accuracy > 50%")
print(f"{better_than_baseline} performed better than the always-up baseline\n")

display(logreg)

Out of 10 tickers:
10 achieved test accuracy > 50%
9 performed better than the always-up baseline



,train_pred_accuracy,test_pred_accuracy,baseline (always positive)
Ticker,,,
NVDA,0.610,0.596,0.551
GOOG,0.546,0.533,0.533
AMZN,0.530,0.522,0.502
AAPL,0.555,0.547,0.512
MSFT,0.544,0.535,0.535
SPOT,0.544,0.512,0.545
TSLA,0.566,0.597,0.493
JPM,0.554,0.589,0.566
GS,0.526,0.522,0.516


### Support Vector Machine

In [12]:
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")

model_dir = "../models"
os.makedirs(model_dir, exist_ok=True)
results = []

for ticker in tickers:
    df = train_dfs[ticker]

    # Features and target
    X = df.drop(columns=["Date", "positive_next_day"])
    y = df["positive_next_day"]

    # Train/test split (80/20)
    split_idx = int(len(df) * 0.8)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    # Time-series cross-validation
    tscv = TimeSeriesSplit(n_splits=5)

    # --- Pipeline ---
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("svc", SVC(probability=True))
    ])

    # --- Hyperparameter grid ---
    grid = {
        "svc__C": [0.1, 1, 5, 10],
        "svc__kernel": ["linear", "rbf"],
        "svc__gamma": ["scale", "auto"]
    }

    # --- Grid search ---
    search = GridSearchCV(
        estimator=pipe,
        param_grid=grid,
        cv=tscv,
        scoring="roc_auc",
        n_jobs=-1
    )
    search.fit(X_train, y_train)

    best_model = search.best_estimator_
    best_params = search.best_params_

    # --- Save the model ---
    model_path = os.path.join(model_dir, f"{ticker}_svc.pkl")
    joblib.dump(best_model, model_path)

    # --- Predictions ---
    train_pred = best_model.predict(X_train)
    test_pred = best_model.predict(X_test)

    # --- Metrics ---
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    test_auc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
    baseline = y_test.mean()  # always predict positive

    results.append({
        "Ticker": ticker,
        "train_pred_accuracy": round(train_acc, 3),
        "test_pred_accuracy": round(test_acc, 3),
        "baseline (always positive)": round(baseline, 3)
    })
# --- Summary Table ---
svc = pd.DataFrame(results).set_index("Ticker")
n_total = len(svc)
above_50 = (svc['test_pred_accuracy'] >= 0.5).sum()
better_than_baseline = (svc['test_pred_accuracy'] >= svc['baseline (always positive)']).sum()

print(f"\nOut of {n_total} tickers:")
print(f"{above_50} achieved test accuracy > 50%")
print(f"{better_than_baseline} performed better than the always-up baseline\n")
svc


Out of 10 tickers:
10 achieved test accuracy > 50%
8 performed better than the always-up baseline



,train_pred_accuracy,test_pred_accuracy,baseline (always positive)
Ticker,,,
NVDA,0.591,0.566,0.551
GOOG,0.589,0.533,0.533
AMZN,0.532,0.532,0.502
AAPL,0.564,0.572,0.512
MSFT,0.527,0.571,0.535
SPOT,0.539,0.529,0.545
TSLA,0.564,0.587,0.493
JPM,0.533,0.566,0.566
GS,0.558,0.554,0.516
